In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# data preprocessing

df = pd.read_excel('Test_dataset.xlsx',sheet_name = 'Put-Call_TS')
df = df.iloc[1:]    # Removing 1st row
df.head()

,Stock Index,Put-Call Ratio,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
1,AC3235,0.8,0.66,0.7724,0.983224,0.901211,0.898368
2,AC3236,0.86,0.732,0.85448,1.08664,1.01704,1.02926
3,AC3237,0.93,0.816,0.95024,1.2073,NaN,1.18196
4,AC3238,1.09,1.008,1.16912,1.48309,1.46106,1.531
5,AC3239,NaN,1.104,1.27856,1.62099,1.6155,1.70552


In [17]:
stock_index = df['Stock Index']

In [19]:
df.head()

,Stock Index,Put-Call Ratio,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
1,AC3235,0.8,0.66,0.7724,0.983224,0.901211,0.898368
2,AC3236,0.86,0.732,0.85448,1.08664,1.01704,1.02926
3,AC3237,0.93,0.816,0.95024,1.2073,NaN,1.18196
4,AC3238,1.09,1.008,1.16912,1.48309,1.46106,1.531
5,AC3239,NaN,1.104,1.27856,1.62099,1.6155,1.70552


In [20]:
df.isnull().sum()   # check for null value

Stock Index         0
Put-Call Ratio    429
Unnamed: 2          2
Unnamed: 3         57
Unnamed: 4         14
Unnamed: 5        105
Unnamed: 6        239
dtype: int64

In [21]:
df["Put-Call Ratio"].fillna(df["Put-Call Ratio"].mean(), inplace=True)
df["Unnamed: 2"].fillna(df["Unnamed: 2"].mean(),inplace=True)
df["Unnamed: 3"].fillna(df["Unnamed: 3"].mean(),inplace=True)
df["Unnamed: 4"].fillna(df["Unnamed: 4"].mean(),inplace=True)
df["Unnamed: 5"].fillna(df["Unnamed: 5"].mean(),inplace=True)
df["Unnamed: 6"].fillna(df["Unnamed: 6"].mean(),inplace=True)



In [35]:
x = df[['Put-Call Ratio','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5']]
y = df['Unnamed: 6']
print(x.shape, y.shape)

(3331, 5) (3331,)


In [36]:
test_x = df[['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6']]

In [37]:
import xgboost as xg

In [49]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
MSE = mean_squared_error
# x = x.to_numpy()
# y = y.to_numpy()
xgb_r = xg.XGBRegressor(objective ='reg:linear', n_estimators = 10, seed = 123) 

xgb_r.fit(x, y)

# xg_preds = xgb_r.predict(test_x)
# print("MSE : ",MSE(test_y, xg_preds), "\nR^2 : ",r2_score(test_y,xg_preds),"\nmae : ",mean_absolute_error(test_y, xg_preds))

[10:20:43] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=8, num_parallel_tree=1,
             objective='reg:linear', random_state=123, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=123, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [57]:
xg_preds = xgb_r.predict(test_x.to_numpy())

In [55]:
print("MSE : ",MSE(y, xg_preds), "\nR^2 : ",r2_score(y,xg_preds),"\nmae : ",mean_absolute_error(y, xg_preds))#measured with train data and changed the value in xg_preds .

MSE :  0.008347110409200761 
R^2 :  0.9539028270826 
mae :  0.046593433365000395


In [58]:
stock_index

1       AC3235
2       AC3236
3       AC3237
4       AC3238
5       AC3239
         ...  
3327    AD2734
3328    AD2735
3329    AD2736
3330    AD2737
3331    AD2738
Name: Stock Index, Length: 3331, dtype: object

In [59]:
pred_df = stock_index.to_frame(name = 'Stock Index')
pred_df['Stock Price']=xg_preds
pred_df

,Stock Index,Stock Price
1,AC3235,0.613008
2,AC3236,1.129945
3,AC3237,1.263498
4,AC3238,1.471592
5,AC3239,1.769298
...,...,...
3327,AD2734,1.129945
3328,AD2735,0.463232
3329,AD2736,0.463232
3330,AD2737,0.577960


In [60]:
pred_df.to_csv('task_2.csv')